# Pengolahan Citra GLCM

## Depedensi

In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [34]:
!pip install scikit-image gradio

import os
import cv2
import numpy as np
import pandas as pd
import gradio as gr
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.color import rgb2gray

## Load Dataset

In [35]:
# Dataset path
image_dir = "/content/drive/MyDrive/Meat Dataset/"

# Load images and labels
def load_images_labels(image_dir):
    images, labels = [], []
    for label in os.listdir(image_dir):
        folder = os.path.join(image_dir, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(label)
    return images, labels

images, labels = load_images_labels(image_dir)
print("Jumlah gambar:", len(images))


Jumlah gambar: 366


## Ekstraksi Fitur

In [36]:
# Cell 4: Ekstraksi Fitur GLCM & LBP
def extract_glcm_features(gray_img):
    glcm = graycomatrix(gray_img, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    features = {
        'contrast': graycoprops(glcm, 'contrast')[0][0],
        'dissimilarity': graycoprops(glcm, 'dissimilarity')[0][0],
        'homogeneity': graycoprops(glcm, 'homogeneity')[0][0],
        'ASM': graycoprops(glcm, 'ASM')[0][0],
        'energy': graycoprops(glcm, 'energy')[0][0],
        'correlation': graycoprops(glcm, 'correlation')[0][0]
    }
    return features

# Contoh uji coba ekstraksi fitur pertama
gray = cv2.cvtColor(images[0], cv2.COLOR_BGR2GRAY)
print("GLCM:", extract_glcm_features(gray))

GLCM: {'contrast': np.float64(308.3304010826772), 'dissimilarity': np.float64(9.00079970472441), 'homogeneity': np.float64(0.17268423877665046), 'ASM': np.float64(0.0005562093033365754), 'energy': np.float64(0.023584090046821295), 'correlation': np.float64(0.7767123981126108)}


## Dataset Fitur

In [38]:
# Cell 5: Buat dataset fitur
def build_feature_dataset(images, labels, feature_type='GLCM'):
    data = []
    for img, label in zip(images, labels):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if feature_type == 'GLCM':
            features = extract_glcm_features(gray)
        else:
            raise ValueError("Fitur tidak dikenali.")
        features['label'] = label
        data.append(features)
    return pd.DataFrame(data)

feature_type = 'GLCM'
df_features = build_feature_dataset(images, labels, feature_type)
df_features.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation,label
0,308.330401,9.000800,0.172684,0.000556,0.023584,0.776712,Horse Meat
1,186.963583,7.908588,0.165549,0.000639,0.025279,0.827457,Horse Meat
2,260.146038,10.204232,0.133256,0.000344,0.018552,0.862162,Horse Meat
3,55.645300,5.333292,0.186634,0.000548,0.023417,0.962476,Horse Meat
4,548.632874,12.518578,0.128469,0.000438,0.020927,0.533505,Horse Meat


## Data Training

In [39]:
# Cell 6: Train-Test Split dan Pelatihan Model
X = df_features.drop('label', axis=1)
y = df_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pilihan model: 'RF' untuk Random Forest, 'SVM' untuk Support Vector Machine
model_choice = 'RF'

if model_choice == 'RF':
    model = RandomForestClassifier(n_estimators=100, random_state=42)
elif model_choice == 'SVM':
    model = SVC(kernel='rbf', probability=True)
else:
    raise ValueError("Model tidak dikenali.")

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Akurasi:", accuracy_score(y_test, y_pred))
print("Laporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.6216216216216216
Laporan Klasifikasi:
               precision    recall  f1-score   support

  Horse Meat       0.60      0.68      0.64        31
        Meat       0.75      0.88      0.81        17
        Pork       0.53      0.38      0.44        26

    accuracy                           0.62        74
   macro avg       0.63      0.65      0.63        74
weighted avg       0.61      0.62      0.61        74



## Gradio Start!

In [42]:
def predict_image(img, model_tipe='Random Forest'):
    img = cv2.resize(img, (128, 128))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Langsung ekstraksi GLCM
    feat_dict = extract_glcm_features(gray)

    # Buat dataframe fitur
    feat_df = pd.DataFrame({
        "📌 Nama Fitur": list(feat_dict.keys()),
        "📈 Nilai": [round(v, 4) for v in feat_dict.values()]
    })

    # Pilih model
    if model_tipe == 'Random Forest':
        mdl = RandomForestClassifier(n_estimators=100, random_state=42)
    else:
        mdl = SVC(kernel='rbf', probability=True)

    # Training
    mdl.fit(X_train, y_train)
    pred = mdl.predict(pd.DataFrame([feat_dict]))[0]
    prob = mdl.predict_proba(pd.DataFrame([feat_dict])).max()

    # Penjelasan
    keterangan = """
## 📘 Penjelasan Fitur Tekstur & Algoritma

### 🧬 **GLCM (Gray Level Co-occurrence Matrix):**
- **Contrast**: Intensitas perbedaan antar piksel.
- **Dissimilarity**: Versi sensitif dari contrast.
- **Homogeneity**: Tekstur halus jika nilainya tinggi.
- **ASM**: Stabilitas tekstur; pola berulang.
- **Energy**: Akar dari ASM.
- **Correlation**: Hubungan antar piksel.

---

### 🧠 **Algoritma:**
- **Random Forest**: Voting dari banyak pohon keputusan. Kuat dan tidak mudah overfitting.
- **SVM (Support Vector Machine)**: Pemisah optimal antar kelas. Akurat untuk data kecil-menengah.
"""
    return (
        f"🧠 Prediksi: {pred}",
        f"📊 Probabilitas: {prob:.2f}",
        feat_df,
        keterangan
    )

In [43]:
# TAMPILAN GRADIO
demo = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.Image(type="numpy", label="📤 Upload Gambar Daging"),
        gr.Radio(['Random Forest', 'SVM'], label="🤖 Pilih Metode")
    ],
    outputs=[
        gr.Textbox(label="🎯 Hasil Prediksi"),
        gr.Textbox(label="📊 Probabilitas"),
        gr.Dataframe(label="📑 Nilai Ekstraksi Fitur"),
        gr.Markdown(label="📘 Penjelasan")
    ],
    title="🥩💻 Klasifikasi Tekstur Citra Daging",
    description="Analisis tekstur menggunakan GLCM dan algoritma Random Forest atau SVM. Output berisi hasil prediksi, nilai fitur, dan penjelasan lengkap biar kamu makin paham 💕",
)

# 🚀 Jalankan interface
demo.launch(share=True, inline=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2d903401e05ac73857.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Evaluasi Model

In [ ]:
# Cell 8: Evaluasi Model Lengkap + Visualisasi
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Prediksi ulang biar sinkron (kalau belum diprediksi di atas)
y_pred = model.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))
report = classification_report(y_test, y_pred, target_names=np.unique(y), output_dict=True)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=np.unique(y), yticklabels=np.unique(y), cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plot Akurasi Per Kelas
plt.figure(figsize=(10, 5))
scores = [report[label]['f1-score'] for label in np.unique(y)]
sns.barplot(x=np.unique(y), y=scores, palette="magma")
plt.title("F1-Score per Class")
plt.ylabel("F1-Score")
plt.xlabel("Class Label")
plt.ylim(0, 1.1)
plt.show()

# Tampilkan laporan klasifikasi
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))